# Check calculator output

In [1]:
# Set up
import pandas as pd
import numpy as np
import sys
from pathlib import Path
CODE_ROOT = Path.cwd().parents[1]
sys.path.append(str(CODE_ROOT))
import config
from openpyxl import load_workbook
from openpyxl.formatting.rule import FormulaRule
from openpyxl.styles import Font, PatternFill
from openpyxl.utils import range_boundaries
import os
import shutil
import xlwings as xw
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

In [2]:
# Load data
labs = pd.read_csv(config.BL_RAW_SAMPLE / "final_sample_with_BL_file_status.csv")

In [3]:
# List of labs to create calculators for (filled files and treatment lab groups only)
treated_labs = labs[(labs["file_filled"] == True) & (labs["Treatment Status"] == "treatment")]

# List to store results
results = []

# Launch excel application
app = xw.App(visible=False)

# For each labgroupid, create a calculator and save in individualized folder
for labgroupid, group in treated_labs.groupby("labgroupid"):

    # Survey file path
    calculator = os.path.join(
        config.DATA_ROOT, 
        "13_Calculators",
        "2_Individualized",
        str(labgroupid),
        "Energy_Use_Report.xlsx"
    )

    if not os.path.exists(calculator):
        print(f"Missing calculator for {labgroupid}")
        continue


    # Load calculator workbook
    calc_wb = xw.Book(calculator)

    # Read total energy use and carbon emissions
    calc_ws = calc_wb.sheets["Introduction"]
    total_energy_use = calc_ws.range("B12").value
    total_carbon_emissions = calc_ws.range("B13").value
    calc_wb.close()

    # Save results 
    results.append({
        "labgroupid": labgroupid,
        "total_energy_use":total_energy_use,
        "total_carbon_emissions": total_carbon_emissions
    })

# Convert to DataFrame
df_results = pd.DataFrame(results)

In [4]:
# Merge with the labs data
labs_send_calcs = treated_labs.merge(df_results, on="labgroupid", how="left")

# Create contact person column which is "Contact person (if different)" if not missing, otherwise "Professor"
labs_send_calcs["Contact person"] = np.where(
    labs_send_calcs["Contact person (if different)"].notna(),
    labs_send_calcs["Contact person (if different)"],
    labs_send_calcs["Professor"]
)

# Create contact email column which is "Contact email (if different)" if not missing, otherwise "Email"
labs_send_calcs["Contact email"] = np.where(
    labs_send_calcs["Contact email (if different)"].notna(),
    labs_send_calcs["Contact email (if different)"],
    labs_send_calcs["Email"]
)

# Keep only relevant columns
cols_to_keep = [
    "labgroupid", "Contact person", "Contact email",
    "total_energy_use", "total_carbon_emissions"
]

# Save to CSV
labs_send_calcs[cols_to_keep].to_csv(
    config.DATA_ROOT / 
    "13_Calculators" / 
    "4_Email_List" / 
    "labs_to_send_calculators.csv", 
    index=False)